In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt


In [2]:
# Load the dataset
df = pd.read_csv('./epa_air_quality_annual_summary.csv')

# Convert the date column to datetime and set it as the index
df['date_of_last_change'] = pd.to_datetime(df['date_of_last_change'])
df.set_index('date_of_last_change', inplace=True)

# Drop rows with missing values in the 'arithmetic_mean' column
df = df.dropna(subset=['arithmetic_mean'])

C:\Users\Shivraj\AppData\Local\Temp\ipykernel_41512\1695695478.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./epa_air_quality_annual_summary.csv')


In [3]:
# Split the data into training and testing sets (80% for training, 20% for testing)
train_size = int(len(df) * 0.8)
train = df.iloc[0:train_size]
test = df.iloc[train_size:len(df)]

# If you specifically want to select the 'arithmetic_mean' column for both training and testing sets
train_arithmetic_mean = train['arithmetic_mean'].fillna(train['arithmetic_mean'].mean())
test_arithmetic_mean = test['arithmetic_mean']

In [4]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import itertools

# Assuming 'train_arithmetic_mean' is already defined from your previous code

def find_best_arima_model(data, p_range, d_range, q_range, criterion='aic'):
    """
    Find the best ARIMA model based on the specified criterion.

    Parameters:
    - data: The time series data.
    - p_range: Range of p values to consider.p (AutoRegressive part): This is the number of previous time steps that the model uses
     to predict the next value
    - d_range: Range of d values to consider.This is the number of times the data has been differenced to make it stationary.
    - q (Moving Average part): This is the number of lagged forecast errors that the model uses to predict the next value.
    It's like using the mistakes made in past predictions to improve future ones.
    - criterion: The criterion to use for model selection ('aic', 'bic', or 'hqic').

    Returns:
    - The best (p, d, q) combination and the corresponding model.
    """
    best_pdq = None
    best_model = None
    best_criterion_value = np.inf if criterion == 'aic' else -np.inf

    for param in itertools.product(p_range, d_range, q_range):
        try:
            model = ARIMA(data, order=param)
            model_fit = model.fit()

            if criterion == 'aic':
                criterion_value = model_fit.aic
            elif criterion == 'bic':
                criterion_value = model_fit.bic
            elif criterion == 'hqic':
                criterion_value = model_fit.hqic
            else:
                raise ValueError(f"Invalid criterion: {criterion}")

            if (criterion == 'aic' and criterion_value < best_criterion_value) or \
               (criterion == 'bic' and criterion_value < best_criterion_value) or \
               (criterion == 'hqic' and criterion_value < best_criterion_value):
                best_pdq = param
                best_model = model_fit
                best_criterion_value = criterion_value

        except Exception as e:
            print(f"Error fitting model with parameters {param}: {e}")
            continue

    if best_pdq is not None:
        print(f"Best ARIMA{best_pdq} model based on {criterion}: {best_criterion_value}")
        return best_pdq, best_model
    else:
        print("No suitable model found.")
        return None, None

# Define the range of p, d, and q values to consider
p_range = range(0, 4)
d_range = range(0, 4)
q_range = range(0, 4)

# Find the best ARIMA model based on AIC
best_pdq, best_model = find_best_arima_model(train_arithmetic_mean, p_range, d_range, q_range, criterion='aic')

# If a best model is found, print its summary
if best_model is not None:
    print(best_model.summary())


c:\Users\Shivraj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Shivraj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Shivraj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Shivraj\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is no

In [ ]:
# Make predictions on the testing set
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

# Ensure both are numpy arrays of floats to avoid type errors
test_arithmetic_mean = np.array(test_arithmetic_mean, dtype=float)
predictions = np.array(predictions, dtype=float)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(test_arithmetic_mean, predictions)
print(f'Mean Squared Error: {mse}')

# Calculate Mean Absolute Percentage Error (MAPE)
# To avoid division by zero, we add a small constant to the denominator
epsilon = 1e-5
mape = np.mean(np.abs((test_arithmetic_mean - predictions) / (test_arithmetic_mean + epsilon)))
print(f'Mean Absolute Percentage Error: {mape}')

# Calculate Mean Squared Logarithmic Error (MSLE)
# Ensure all values are strictly positive
test_arithmetic_mean_positive = np.maximum(test_arithmetic_mean, epsilon)
predictions_positive = np.maximum(predictions, epsilon)

msle = mean_squared_error(np.log(test_arithmetic_mean_positive), np.log(predictions_positive))
print(f'Mean Squared Logarithmic Error: {msle}')


In [ ]:
plt.plot(range(len(predictions)), test_arithmetic_mean, c="b", label="True")
plt.plot(range(len(predictions)), predictions, c="r", label="pred")
plt.legend()
plt.show()